In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard library imports.
import datetime
import itertools
import json

# Related third party imports.
import numpy as np
import pandas as pd
import tqdm

# Local application/library specific imports.
from src.models.network_simulation import NetworkSimulation
from src.data.json_encoder import NumpyEncoder

In [3]:
network_name = '25_italy'
max_vSDN_size = 6
hp_type = 'ilp'
hp_objective = 'acceptance ratio'

possible_settings = {
    'network_name': [network_name],
    'latency_factor': [0.5], # np.arange(0.4, 0.7, 0.1),
    'shortest_k': [16],
    'hp_type': [hp_type],
    'hp_objective': [hp_objective],
    'repeat': [100],
    'timesteps': [20],
    'max_request_size': [max_vSDN_size],
    'request_per_timestep':[10],
    'vSDN_count_ilp':[100]
}
param_names_1 = list(possible_settings.keys())
setting_list = [
    dict(zip(param_names_1, x))
    for x in itertools.product(*possible_settings.values())
]

In [6]:
# Run
setting = setting_list[0]

ns = NetworkSimulation(**setting)
ns.init_simulation(**setting)
ns.run_dynamic_simulation(**setting)
run_log = ns.get_logs()
df = pd.DataFrame(run_log)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 75 rows, 8775 columns and 2105 nonzeros
Model fingerprint: 0x35030414
Model has 7550 general constraints
Variable types: 0 continuous, 8775 integer (8775 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve added 2251 rows and 0 columns
Presolve removed 0 rows and 6770 columns
Presolve time: 0.08s
Presolved: 2326 rows, 2005 columns, 10479 nonzeros
Variable types: 0 continuous, 2005 integer (2005 binary)
Found heuristic solution: objective 24.0000000

Root relaxation: objective 4.900000e+00, 1387 iterations, 0.07 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.90000    0  230   24.0000

In [10]:
# run_log = ns.log_simulation()
pd.concat([df[['vSDN_size', 'acceptance_ratio']],df_opt[['acceptance_ratio']]], axis=1)

,vSDN_size,acceptance_ratio,acceptance_ratio
0,4.6,0.9,0.9
1,5.4,0.8,0.8
2,5.4,0.9,0.9
3,4.8,0.7,0.7
4,5.6,1.0,1.0
5,5.0,0.9,0.9
6,5.5,0.8,0.8
7,5.5,0.8,0.8
8,5.2,0.8,0.8
9,4.5,0.8,0.8


In [ ]:
# Simulation
simulation_logs = []
for setting in tqdm.tqdm(setting_generator, total=len(setting_generator)):
    ns = NetworkSimulation(**setting)
    ns.init_simulation(**setting)
    ns.run_simulation(possible_request_settings=possible_request_settings, **setting)
    simulation_logs.extend(ns.get_logs())

In [7]:
# Optimal Run
setting = setting_list[0]

ns = NetworkSimulation(**setting)
ns.init_simulation(**setting)
ns.run_optimal_dynamic_simulation(**setting)
run_log_opt = ns.get_logs()
df_opt = pd.DataFrame(run_log_opt)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 75 rows, 8775 columns and 2105 nonzeros
Model fingerprint: 0x35030414
Model has 7550 general constraints
Variable types: 0 continuous, 8775 integer (8775 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve added 2251 rows and 0 columns
Presolve removed 0 rows and 6770 columns
Presolve time: 0.08s
Presolved: 2326 rows, 2005 columns, 10479 nonzeros
Variable types: 0 continuous, 2005 integer (2005 binary)
Found heuristic solution: objective 24.0000000

Root relaxation: objective 4.900000e+00, 1387 iterations, 0.07 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.90000    0  230   24.0000

In [8]:
df_opt[['vSDN_size', 'acceptance_ratio']]

,vSDN_size,acceptance_ratio
0,4.6,0.9
1,5.4,0.8
2,5.4,0.9
3,4.8,0.7
4,5.6,1.0
5,5.0,0.9
6,5.5,0.8
7,5.5,0.8
8,5.2,0.8
9,4.5,0.8


In [ ]:
# Baseline Simulation
simulation_logs = []
for setting in tqdm.tqdm(setting_generator, total=len(setting_generator)):
    ns = NetworkSimulation(**setting)
    ns.init_simulation(**setting)
    ns.baseline_simulation(possible_request_settings=possible_request_settings, **setting)
    simulation_logs.extend(ns.get_logs())

In [ ]:
with open(f"../results/{network_name}/{datetime.date.today()}-{network_name}-test.json", 'w') as file:
    json.dump(simulation_logs, file, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)

# Analyse simulation logs

In [ ]:
df_heu = pd.DataFrame(simulation_logs)

In [ ]:
df_heu.query("latency_factor == 0.5 & vSDN_size == 2")[['hp_runtime', 'acceptance_ratio']]#.mean()

In [5]:
df_ilp = pd.DataFrame(simulation_logs)

In [6]:
df_ilp.query("latency_factor == 0.4 & vSDN_size == 3")[['hp_runtime', 'acceptable_count','acceptance_ratio', 'acceptance_ratio_ilp', 'h_list']]#.mean()

,hp_runtime,acceptable_count,acceptance_ratio,acceptance_ratio_ilp,h_list
1,0.73,36,0.580645,None,"[3, 6, 12, 15, 18, 19, 20, 23]"
9,0.61,33,0.532258,None,"[3, 6, 12, 15, 18, 19, 20, 23]"
17,0.64,32,0.516129,None,"[3, 6, 12, 15, 18, 19, 20, 23]"
25,0.69,31,0.500000,None,"[3, 6, 12, 15, 18, 19, 20, 23]"


In [ ]:
df_opt = pd.DataFrame(simulation_logs)

In [ ]:
df_opt.query("latency_factor == 0.4 & vSDN_size == 2")[['hp_runtime', 'acceptance_ratio', 'acceptance_ratio_ilp']]#.mean()